# ewx_public

In [1]:
#! pip3 install googleapiclient

In [5]:
from dateutil.parser import parse
import logging
import os
import json
import socket
import requests
import time
import re
import urllib
from retrying import retry
import datetime as dt
import pprint

#from googleapiclient.errors import HttpError

#from auth import DEFAULT_API_ROOT
from energyworx_public import domain, rule
from energyworx_public.base import EnergyworxDomain, StructuredProperty, Property, EnumProperty, DateTimeProperty, MappingStructuredProperty, MappingProperty, MappingEnumProperty
from energyworx_public.enums import MappedFieldType, TransformationMapFunctionType, TagType, UnitType, VirtualDatasourceAggregationType, DatasourceType, DatapointType, TimeslicePeriodType

from energyworx_client import client
#from tags import parse_tags
#from timeseries import parse_result_df, parse_nested_structure_result_df

logger = logging.getLogger()
pp = pprint.PrettyPrinter(1)
print('imported modules successfully, {}.'.format(dt.datetime.today()))

imported modules successfully, 2019-12-09 18:12:03.151025.


## Request Headers

In [83]:
namespace = 'na.engie.com'
default_api_root = 'https://ewx-live.appspot.com/_ah/api/'
other = 'cloud.app.energyworx.com/_ah/api'

creds = {}
bearer_token = 'ya29.ImC0BxhNTmgL1ZjjrNfX0NxIWmbj4D99HzS-Y2w-2-SjA1gocZ1xnno-gnU7dn-Ooi2S-25Ek2h7JzGtnYPEeO6tFs19_r44pqh5i5gFw3Iq97rm9ykhztQFMyPr036haOA'
auth = ''.join(['Bearer ', bearer_token])

header = {'Authorization':auth, 'X-NAMESPACE':namespace, 'accept':'application/json'}

# X_API_KEY="626E2AA48CE14D5EA2B4DB8F2052B9FC"
pp.pprint(header)

{'Authorization': 'Bearer '
                  'ya29.ImC0BxhNTmgL1ZjjrNfX0NxIWmbj4D99HzS-Y2w-2-SjA1gocZ1xnno-gnU7dn-Ooi2S-25Ek2h7JzGtnYPEeO6tFs19_r44pqh5i5gFw3Iq97rm9ykhztQFMyPr036haOA',
 'X-NAMESPACE': 'na.engie.com',
 'accept': 'application/json'}


In [84]:
response = requests.get('https://ewx-live.appspot.com/_ah/api/ewx/v1/datasource/get/NEPOOL_BECO_26701281052?alt=json', params = header)

In [85]:
response.json()

{'error': {'code': 401,
  'errors': [{'domain': 'global',
    'message': 'Invalid token.',
    'reason': 'required'}],
  'message': 'Invalid token.'}}

## Base properties

In [6]:
### name=None, required=False, validator=None, repeated=False, default=None
Property()

#inherits from properties:
DateTimeProperty()
EnumProperty(0) #enum_type (0 - kWh, 12 - seconds)
StructuredProperty(domain.Channel()) #domain_class (Channel, ChannelClassifier, Tag, KeyValue, KeyValueType)
MappingProperty()

#inherits from EnumProp
MappingEnumProperty(0)

#inherits from StructProp
MappingStructuredProperty(domain.Channel())

## Domain Metaclass

In [7]:
class EnergyworxDomainMeta(type):
    """
    A metaclass which is used to add extra attributes 'attr_by_prop_name' and 'prop_by_attr' to every domain class
    """
    def __new__(mcs, name, bases, dct):
        new_class = super(EnergyworxDomainMeta, mcs).__new__(mcs, name, bases, dct)
        attr_by_prop_name = {}
        prop_by_attr = {}
        for attr, value in dct.items():
            if isinstance(value, Property):
                setattr(value, 'attr_name', attr)
                attr_by_prop_name.update({value.name: attr})
                prop_by_attr.update({attr: value})

        if attr_by_prop_name:
            new_class.attr_by_prop_name = attr_by_prop_name

        if prop_by_attr:
            new_class.prop_by_attr = prop_by_attr

        return new_class

## Domain Class

In [8]:
class EnergyworxDomain(object):

    """ Base class for every domain class """

    __metaclass__ = EnergyworxDomainMeta

    id = Property(name="id", required=True)
    read_only = Property(name="readOnly", default=False)

    def __init__(self, **kwargs):
        for attr, value in kwargs.items():
            setattr(self, attr, value)
            
    @classmethod
    def from_message(cls, message):
        """
        Create domain object from a protojson message
        Args:
            message(dict(str, str)): proto json

        Returns:
            (EnergyworxDomain): a domain object
        """
        kwargs = {}
        for prop_name, value in message.items():
            logger.debug('property {prop_name}, value: {value}'.format(prop_name=prop_name, value=value))
            attr = cls.attr_by_prop_name.get(prop_name) or (super(cls, cls).attr_by_prop_name.get(prop_name))
            if not attr:
                raise AttributeError('Property name {prop_name} can not be mapped to a domain attribute on domain class {class_name}'.format(prop_name=prop_name, class_name=cls.__name__))
            prop = cls.prop_by_attr.get(attr) or (super(cls, cls).prop_by_attr.get(attr))
            if not prop:
                raise AttributeError('Attribute {attr_name} can not be mapped to a property instance on domain class {class_name}'.format(attr_name=attr, class_name=cls.__name__))

            if isinstance(prop, StructuredProperty):
                if prop.repeated:
                    kwargs.update({attr: [prop.domain_class.from_message(v) for v in value]})
                else:
                    kwargs.update({attr: prop.domain_class.from_message(value)})
            elif isinstance(prop, EnumProperty):
                kwargs.update({attr: str_to_enum(prop.enum_type, value)})
            elif isinstance(prop, DateTimeProperty):
                kwargs.update({attr: parse(value)})
            else:
                kwargs.update({attr: value})
        return cls(**kwargs)
    
    
    def to_message(self):
        """
        Create a dict based on property name and attribute
        Returns:
            (dict): a dict based on property name and attribute value
        """
        prop_by_attr = {}
        prop_by_attr.update(self.prop_by_attr.items())
        if not prop_by_attr.get('id') and EnergyworxDomain in self.__class__.__bases__:
            prop_by_attr['id'] = super(self.__class__, self).prop_by_attr.get('id')

        message = {}
        for attr, prop in prop_by_attr.items():
            if isinstance(prop, StructuredProperty):
                if prop.repeated:
                    message.update({prop.name: [item.to_message() for item in getattr(self, attr)]})
                else:
                    message.update({prop.name: getattr(self, attr).to_message()})
            elif isinstance(prop, EnumProperty):
                message.update({prop.name: getattr(self, attr).name if getattr(self, attr) else None})
            elif isinstance(prop, DateTimeProperty):
                message.update({prop.name: getattr(self, attr).replace(tzinfo=None).strftime('%Y-%m-%dT%H:%M:%S.%f') if getattr(self, attr) else None})
            elif getattr(self, attr):
                message.update({prop.name: getattr(self, attr)})
        return message

## Key Value

In [9]:
class KeyValue(EnergyworxDomain):
    key = Property(name="key", required=True)
    value = Property(name="value", required=True)
    deleted = Property(name="deleted", default=False)

    def to_dict(self):
        return dict(key=self.key,
                    value=self.value,
                    read_only=self.read_only)
    @classmethod
    def from_dict(cls, keyvalue_dict):
        return KeyValue(key=keyvalue_dict.get('key'),
                        value=keyvalue_dict.get('value'),
                        read_only=keyvalue_dict.get('read_only'))


class KeyValueType(KeyValue):
    value_type = Property(name="type", required=True, default="String")


## Tag

In [10]:
class Tag(EnergyworxDomain):
    tag = Property(name="tag", required=True)
    description = Property(name="description", default='No description')
    valid_from = DateTimeProperty(name="validFrom")
    properties = StructuredProperty(KeyValueType, name="properties", repeated=True)
    removed = Property(name="removed", default=False)
    version = DateTimeProperty(name="version")
    is_active_scd = Property(name="isActiveSCD", required=False)
    tag_links = Property(name="tagLinks", default=False)
    created_by = Property(name="createdBy", default=None)

    def __init__(self, *args, **kwargs):
        super(Tag, self).__init__(*args, **kwargs)
        if not self.valid_from:
            self.valid_from = datetime(1900,1,1,0,0,0)
        if not self.version:
            self.version = datetime.utcnow()

    def get_property(self, property_key):
        return next((key_value.value for key_value in self.properties if key_value.key == property_key), None)

    def __eq__(self, other):
        if isinstance(other, Tag):
            if self.tag != other.tag:
                return False
            if ((self.valid_from is None and other.valid_from is not None) or
                    (self.valid_from is not None and other.valid_from is None)):
                return False
            if (self.valid_from is not None and other.valid_from is not None and
                    self.valid_from.replace(tzinfo=None) != other.valid_from.replace(tzinfo=None)):
                return False
            if self.description != other.description:
                return False
            if self.removed != other.removed:
                return False
            self_property_dict = {kv.key: kv.value for kv in self.properties}
            other_property_dict = {kv.key: kv.value for kv in other.properties}
            # check whether the two dictionaries are equal
            if self_property_dict != other_property_dict:
                return False
            return True
        else:
            return False

    def to_dict(self):
        return dict(tag=self.tag,
                    tag_links=self.tag_links,
                    description=self.description,
                    validFrom=self.valid_from.strftime(DATETIME_STRING_FORMAT) if self.valid_from else None,
                    properties=[property.to_dict() for property in self.properties],
                    version=self.version.strftime(DATETIME_STRING_FORMAT) if self.version else None,
                    removed=self.removed,
                    read_only=self.read_only,
                    created_by=self.created_by)

    @classmethod
    def from_dict(cls, tag_dict):
        return Tag(tag=tag_dict.get('tag'),
                   tag_links=tag_dict.get('tag_links', []),
                   description=tag_dict.get('description'),
                   valid_from=dateutil.parser.parse(tag_dict.get('validFrom')),
                   properties=[KeyValue.from_dict(keyvalue_dict) for keyvalue_dict in tag_dict.get('properties')] if tag_dict.get('properties') else None,
                   version=dateutil.parser.parse(tag_dict.get('version')),
                   removed=tag_dict.get('removed'),
                   read_only=tag_dict.get('read_only'),
                   created_by=tag_dict.get('created_by'))


## Channel Classifier

In [11]:
class ChannelClassifier(EnergyworxDomain):
    unit_type = EnumProperty(UnitType, name="unitType", required=True)
    datapoint_type = EnumProperty(DatapointType, name="datapointType", required=True)
    default_flow_configuration_id = Property(name="defaultFlowConfigurationId", required=False)
    aggregation_type = EnumProperty(VirtualDatasourceAggregationType, required=False)
    timeslice_group_model_ids = Property(name='timesliceGroupIds', repeated=True)
    default_timeslice_group_model = Property(name='defaultTimesliceId')

## Channel

In [12]:
class Channel(EnergyworxDomain):
    id = Property(name="id", required=True)
    deleted = Property(name="deleted")
    name = Property(name="name")
    classifier = Property(name="classifier", required=True)
    description = Property(name="description", default="No description")
    unit_type = EnumProperty(UnitType, name="unitType")
    datapoint_type = EnumProperty(DatapointType, name="datapointType")
    is_source = Property(name="isSource")
    flow_configuration_id = Property(name="flowConfigurationId", required=False)
    # the next property is only used by the VirtualDatasources
    aggregation_type = EnumProperty(VirtualDatasourceAggregationType, required=False)

    # TODO DELETE: This line can be removed once all clients are migrated
    datasource_type = EnumProperty(DatasourceType, name="datasourceType")

    def __eq__(self, other):
        if isinstance(other, Channel):
            if (self.id != other.id or
                    self.name != other.name or
                    self.description != other.description or
                    self.classifier != other.classifier or
                    self.is_source != other.is_source):
                return False
            if ((self.unit_type is None and other.unit_type is not None) or
                    (self.unit_type is not None and other.unit_type is None)):
                return False
            if (self.unit_type is not None and other.unit_type is not None and
                    self.unit_type.name != other.unit_type.name):
                return False
            if ((self.datapoint_type is None and other.datapoint_type is not None) or
                    (self.datapoint_type is not None and other.datapoint_type is None)):
                return False
            if (self.datapoint_type is not None and other.datapoint_type is not None and
                    self.datapoint_type.name != other.datapoint_type.name):
                return False
            if ((self.aggregation_type is None and other.aggregation_type is not None) or
                    (self.aggregation_type is not None and other.aggregation_type is None)):
                return False
            if (self.aggregation_type is not None and other.aggregation_type is not None and
                    self.aggregation_type.name != other.aggregation_type.name):
                return False
            return True
        else:
            return False

## Datasource Object

In [13]:
class Datasource(EnergyworxDomain):
    channels = StructuredProperty(Channel, name="channels", repeated=True)
    tags = StructuredProperty(Tag, name="tags", repeated=True)
    name = Property(name="name", required=True)
    description = Property(name="description", required=False, default="No description")
    timezone = Property(name="timezone", required=False)
    filter = Property(name="filter", required=False)
    limit = Property(name="limit")
    more = Property(name="more")
    updated_datetime = Property(name="updatedDatetime")
    created_datetime = Property(name="createdDatetime")
    classifier = Property(name="classifier", required=False)

    def get_channel_by_id(self, channel_id):
        for channel in self.channels:
            if channel.id == channel_id:
                return channel

    def get_channel_by_classifier(self, channel_classifier):
        for channel in self.channels:
            if channel.classifier == channel_classifier:

                return channel

    def get_source_channels(self):
        for channel in self.channels:
            if channel.is_source:
                yield channel

    def get_tag(self, tag_name, latest_version=False, active_versions=False, all_versions=False):
        """ Get the tag with name tag_id. You have the option to select
        the latest version, the active versions and/or all versions. A
        tuple with three values will always be returned, however, only
        the parameters with value True will actually return the wanted
        tags. The order of the return values in the returned tuple is:
        latest_version, active_versions, all_versions.

        Args:
            tag_name (str): Name of the tag to be retrieved
            latest_version (bool):
            active_versions (bool):
            all_versions (bool):

        Returns:

            tuple[Tag, list[Tag], list[Tag]]: if a list of tags is returned,
                the list is sorted in asceding order based on tag.version
        """
        corresponding_tags = filter(lambda tag: tag.tag == tag_name, self.tags)  # type: Union[str, unicode, Tuple[Any, ...], List[Any]]
        if not corresponding_tags:
            # no tags found for tag_name
            logger.warn("No tag found for name: %s", tag_name)
            return None, [], []
        # sort the tags based on the version attribute in ascending order
        corresponding_tags = sorted(corresponding_tags, key=operator.attrgetter('version'), reverse=False)
        # prepare latest version response
        latest_version_result = None
        if latest_version:
            latest_version_result = corresponding_tags[-1]
        # prepare active versions response
        active_versions_result = []
        if active_versions:
            # filters out any tag versions that are overruled in the SCD timeline
            valid_froms = [t.valid_from for t in corresponding_tags]
            active_versions_result = []
            latest_active_valid_from = datetime(3000, 1, 1, 1, tzinfo=pytz.UTC)
            for idx in range(-1, -(len(valid_froms) + 1), -1):
                valid_from = valid_froms[idx]
                if valid_from < latest_active_valid_from:
                    # prepend the tag to get a list of tags in ascending order (based on version)
                    active_versions_result.insert(0, corresponding_tags[idx])
                    latest_active_valid_from = valid_from
        # prepare all versions response
        all_versions_result = []
        if all_versions:
            all_versions_result = corresponding_tags
        return latest_version_result, active_versions_result, all_versions_result

    def cache_compare(self, cache_datasource):
        """
        Only compare the available channels and tags against cached datasource ignore channels that are available
        in cached datasource that are not available in this instance
        """
        return cache_datasource is not None and all([cache_datasource is not None,
                                                     self.id == cache_datasource.id,
                                                     self.name == cache_datasource.name,
                                                     self.description == cache_datasource.description,
                                                     self.classifier == cache_datasource.classifier,
                                                     self.timezone == cache_datasource.timezone,
                                                     self.filter == cache_datasource.filter,
                                                     all([channel == cache_datasource.get_channel_by_id(channel.id) for channel in self.channels]),
                                                     all([any([tag == version for version in cache_datasource.get_tag(tag.tag, all_versions=True)[2]]) for tag in self.tags])])

    def __eq__(self, other):
        if isinstance(other, Datasource):
            if (self.id != other.id or
                    self.name != other.name or
                    self.description != other.description or
                    self.classifier != other.classifier or
                    self.timezone != other.timezone):
                return False
            if len(self.tags) != len(other.tags):
                return False
            for _tag in self.tags:
                if _tag not in other.tags:
                    return False
            if len(self.channels) != len(other.channels):
                return False
            for _channel in self.channels:
                if _channel not in other.channels:
                    return False
            return True
        else:
            return False

## Datasource from message
load from json, create domain obj, datasource obj, datasource from message

In [23]:
ds = json.loads('{"description": "", "tags": [{"validFrom": "1900-01-01T00:00:00.000000", "description": "", "tagLinks": [""], "version": "2019-12-03T19:16:17.000000", "tag": "premise", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "MARLBOROUGH", "readOnly": false, "value_type": "string", "key": "city", "type": "string"}, {"deleted": false, "value": "TIERPOINT-MA", "readOnly": false, "value_type": "string", "key": "name", "type": "string"}, {"deleted": false, "value": "01752", "readOnly": false, "value_type": "string", "key": "zip", "type": "string"}, {"deleted": false, "value": "USA", "readOnly": false, "value_type": "string", "key": "country", "type": "string"}, {"deleted": false, "value": "Unknown", "readOnly": false, "value_type": "string", "key": "county", "type": "string"}, {"deleted": false, "value": "MA", "readOnly": false, "value_type": "string", "key": "state", "type": "string"}, {"deleted": false, "value": "34 SAINT MARTIN DR UNIT 22 UNKNOWN", "readOnly": false, "value_type": "string", "key": "address", "type": "string"}, {"deleted": false, "value": "1-XWPK9R", "readOnly": false, "value_type": "string", "key": "customerid", "type": "string"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "icap", "tagLinks": [""], "version": "2017-11-01T19:47:48.000000", "tag": "icap_utility_2018_06_01", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "calculated", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "default", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "estimated", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "passed", "type": "String"}, {"deleted": false, "value": "1830.78", "readOnly": false, "value_type": "String", "key": "peak", "type": "String"}, {"deleted": false, "value": "2017-09-01 00:00:00+00:00", "readOnly": false, "value_type": "String", "key": "peakindex", "type": "String"}, {"deleted": false, "value": "-93.8438261288", "readOnly": false, "value_type": "String", "key": "percentagediff", "type": "String"}, {"deleted": false, "value": "S", "readOnly": false, "value_type": "String", "key": "periodstatus", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "utilityprovided", "type": "String"}, {"deleted": false, "value": "112.706", "readOnly": false, "value_type": "String", "key": "value", "type": "String"}, {"deleted": false, "value": "2019-05-31 23:59:59", "readOnly": false, "value_type": "String", "key": "valid_till", "type": "String"}]}, {"validFrom": "2019-06-01T00:00:00.000000", "description": "No description", "tagLinks": [""], "version": "2019-12-03T19:16:18.000000", "tag": "geopoint", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "42.3129021", "readOnly": false, "value_type": "String", "key": "latitude", "type": "String"}, {"deleted": false, "value": "-71.57925709999999", "readOnly": false, "value_type": "String", "key": "longitude", "type": "String"}, {"deleted": false, "value": "34 saint martin dr unit 22 , marlborough, usa, ma, 01752", "readOnly": false, "value_type": "String", "key": "address", "type": "String"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "icap", "tagLinks": [""], "version": "2017-11-01T19:47:48.000000", "tag": "icap_utility_2017_06_01", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "calculated", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "default", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "estimated", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "passed", "type": "String"}, {"deleted": false, "value": "1746.675", "readOnly": false, "value_type": "String", "key": "peak", "type": "String"}, {"deleted": false, "value": "2017-05-18 16:00:00+00:00", "readOnly": false, "value_type": "String", "key": "peakindex", "type": "String"}, {"deleted": false, "value": "-93.5473971975", "readOnly": false, "value_type": "String", "key": "percentagediff", "type": "String"}, {"deleted": false, "value": "H", "readOnly": false, "value_type": "String", "key": "periodstatus", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "utilityprovided", "type": "String"}, {"deleted": false, "value": "112.706", "readOnly": false, "value_type": "String", "key": "value", "type": "String"}, {"deleted": false, "value": "2018-05-31 23:59:59", "readOnly": false, "value_type": "String", "key": "valid_till", "type": "String"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "icap", "tagLinks": [""], "version": "2017-11-01T19:47:48.000000", "tag": "icap_utility_2019_06_01", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "calculated", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "default", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "estimated", "type": "String"}, {"deleted": false, "value": "S", "readOnly": false, "value_type": "String", "key": "periodstatus", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "utilityprovided", "type": "String"}, {"deleted": false, "value": "112.706", "readOnly": false, "value_type": "String", "key": "value", "type": "String"}, {"deleted": false, "value": "2029-05-31 23:59:59", "readOnly": false, "value_type": "String", "key": "valid_till", "type": "String"}]}, {"validFrom": "2018-06-01T00:00:00.000000", "description": "", "tagLinks": [""], "version": "2019-12-03T19:16:17.000000", "tag": "transactioninfo", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "ENGIE", "readOnly": false, "value_type": "String", "key": "customername", "type": "String"}, {"deleted": false, "value": "predealveepayload", "readOnly": false, "value_type": "string", "key": "payloadtype", "type": "string"}, {"deleted": false, "value": "2019-12-03 13:11:00", "readOnly": false, "value_type": "string", "key": "creationdate", "type": "string"}, {"deleted": false, "value": "1-XWPK9R", "readOnly": false, "value_type": "string", "key": "customerid", "type": "string"}, {"deleted": false, "value": "20191203131100642000_NEPOOL_MECO_0219222018", "readOnly": false, "value_type": "string", "key": "referenceid", "type": "string"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "No description", "tagLinks": [""], "version": "2017-10-26T13:45:18.000000", "tag": "run_config", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "5629499534213120", "readOnly": false, "value_type": "String", "key": "delivery_idr", "type": "String"}, {"deleted": false, "value": "5707702298738688", "readOnly": false, "value_type": "String", "key": "delivery_scalar", "type": "String"}]}, {"validFrom": "2018-06-01T00:00:00.000000", "description": "No description", "tagLinks": [""], "version": "2017-11-10T18:32:08.000000", "tag": "datasourcegroup", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "1-FRWY3E_1", "readOnly": false, "value_type": "String", "key": "datasourcegroupid", "type": "String"}]}, {"validFrom": "2017-06-01T00:00:00.000000", "description": "No description", "tagLinks": [""], "version": "2017-11-01T19:47:48.000000", "tag": "datasource", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "default", "readOnly": false, "value_type": "String", "key": "datasource_type", "type": "String"}, {"deleted": false, "value": "Automatic generated datasource", "readOnly": false, "value_type": "String", "key": "description", "type": "String"}, {"deleted": false, "value": "NEPOOL_MECO_0219222018", "readOnly": false, "value_type": "String", "key": "id", "type": "String"}, {"deleted": false, "value": "NEPOOL_MECO_0219222018", "readOnly": false, "value_type": "String", "key": "name", "type": "String"}, {"deleted": false, "value": "America/New_York", "readOnly": false, "value_type": "String", "key": "timezone", "type": "String"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "icap", "tagLinks": [""], "version": "2017-11-01T22:21:16.000000", "tag": "icap_calculated_2019_06_01", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "calculated", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "default", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "estimated", "type": "String"}, {"deleted": false, "value": "E", "readOnly": false, "value_type": "String", "key": "periodstatus", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "utilityprovided", "type": "String"}, {"deleted": false, "value": "1793.8759986", "readOnly": false, "value_type": "String", "key": "value", "type": "String"}, {"deleted": false, "value": "2020-05-31 23:59:59", "readOnly": false, "value_type": "String", "key": "valid_till", "type": "String"}]}, {"validFrom": "2019-06-01T00:00:00.000000", "description": "icap", "tagLinks": [""], "version": "2017-11-01T22:21:16.000000", "tag": "icap_calculated_2018_06_01", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "-2.01575292498", "readOnly": false, "value_type": "String", "key": "percentagediff", "type": "String"}, {"deleted": false, "value": "2017-09-01 00:00:00+00:00", "readOnly": false, "value_type": "String", "key": "peakindex", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "passed", "type": "String"}, {"deleted": false, "value": "1830.78", "readOnly": false, "value_type": "String", "key": "peak", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "calculated", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "default", "type": "String"}, {"deleted": false, "value": "True", "readOnly": false, "value_type": "String", "key": "estimated", "type": "String"}, {"deleted": false, "value": "E", "readOnly": false, "value_type": "String", "key": "periodstatus", "type": "String"}, {"deleted": false, "value": "False", "readOnly": false, "value_type": "String", "key": "utilityprovided", "type": "String"}, {"deleted": false, "value": "1793.8759986", "readOnly": false, "value_type": "String", "key": "value", "type": "String"}, {"deleted": false, "value": "2019-05-31 23:59:59", "readOnly": false, "value_type": "String", "key": "valid_till", "type": "String"}]}, {"validFrom": "1900-01-01T00:00:00.000000", "description": "", "tagLinks": [""], "version": "2019-12-03T19:16:17.000000", "tag": "metadata", "createdBy": "System", "removed": false, "properties": [{"deleted": false, "value": "0219222018", "readOnly": false, "value_type": "string", "key": "ldc_account_number", "type": "string"}, {"deleted": false, "value": "MECO", "readOnly": false, "value_type": "string", "key": "discocode", "type": "string"}, {"deleted": false, "value": "NEPOOL", "readOnly": false, "value_type": "string", "key": "market", "type": "string"}, {"deleted": false, "value": "WCMASS", "readOnly": false, "value_type": "string", "key": "zone", "type": "string"}, {"deleted": false, "value": "NEPOOL_MECO_0219222018", "readOnly": false, "value_type": "string", "key": "accountnumber", "type": "string"}, {"deleted": false, "value": "KWH", "readOnly": false, "value_type": "string", "key": "unittype", "type": "string"}, {"deleted": false, "value": "PRIMARY", "readOnly": false, "value_type": "string", "key": "lossclass", "type": "string"}, {"deleted": false, "value": "IDR", "readOnly": false, "value_type": "string", "key": "profileclass", "type": "string"}, {"deleted": false, "value": "I", "readOnly": false, "value_type": "string", "key": "profilebuilder", "type": "string"}, {"deleted": false, "value": "IDR", "readOnly": false, "value_type": "string", "key": "metertype", "type": "string"}, {"deleted": false, "value": "PRE_DEAL", "readOnly": false, "value_type": "string", "key": "stageidentifier", "type": "string"}, {"deleted": false, "value": "1-XWPKA5", "readOnly": false, "value_type": "string", "key": "prnumber", "type": "string"}, {"deleted": false, "value": "KW", "readOnly": false, "value_type": "string", "key": "totaldemandunittype", "type": "string"}, {"deleted": false, "value": "Y", "readOnly": false, "value_type": "string", "key": "forecastflag", "type": "string"}, {"deleted": false, "value": "17", "readOnly": false, "value_type": "string", "key": "meterreadcycle", "type": "string"}, {"deleted": false, "value": "G3", "readOnly": false, "value_type": "string", "key": "rateclass", "type": "string"}]}], "id": "NEPOOL_MECO_0219222018", "channels": [{"description": "Automatic generated channel", "isSource": true, "classifier": "DELIVERY_IDR", "unitType": "kWh", "datapointType": "interval", "id": "DELIVERY_IDR", "name": "Delivery Idr"}, {"description": "Automatic generated channel", "isSource": true, "classifier": "DELIVERY_SCALAR", "unitType": "kWh", "datapointType": "gauge", "id": "DELIVERY_SCALAR", "name": "Delivery Scalar"}, {"description": "No description", "isSource": false, "classifier": "FORECAST", "unitType": "kWh", "datapointType": "interval", "id": "FORECAST", "name": "No name"}, {"description": "No description", "isSource": false, "classifier": "LINEAR_INTERPOLATION", "unitType": "kWh", "datapointType": "interval", "id": "LINEAR_INTERPOLATION", "name": "No name"}, {"description": "No description", "isSource": false, "classifier": "LIKE_DAY_SUBSTITUTION", "unitType": "kWh", "datapointType": "interval", "id": "LIKE_DAY_SUBSTITUTION", "name": "No name"}, {"description": "No description", "isSource": false, "classifier": "MERGED", "unitType": "kWh", "datapointType": "gauge", "id": "MERGED", "name": "No name"}, {"name": "DELIVERY_SCALAR_HEARTBEAT", "isSource": false, "classifier": "DELIVERY_SCALAR_HEARTBEAT", "unitType": "seconds", "datapointType": "gauge", "id": "DELIVERY_SCALAR_HEARTBEAT"}, {"description": "", "isSource": true, "classifier": "TOTAL_DEMAND", "unitType": "kWh", "datapointType": "gauge", "id": "TOTAL_DEMAND", "name": "TOTAL_DEMAND"}, {"name": "TOTAL_DEMAND_HEARTBEAT", "isSource": false, "classifier": "TOTAL_DEMAND_HEARTBEAT", "unitType": "seconds", "datapointType": "gauge", "id": "TOTAL_DEMAND_HEARTBEAT"}, {"name": "DELIVERY_IDR_HEARTBEAT", "isSource": false, "classifier": "DELIVERY_IDR_HEARTBEAT", "unitType": "seconds", "datapointType": "gauge", "id": "DELIVERY_IDR_HEARTBEAT"}, {"description": "", "isSource": true, "classifier": "ICAP", "unitType": "kWh", "datapointType": "gauge", "id": "ICAP", "name": "Installed Capacity"}, {"name": "Installed Capacity_HEARTBEAT", "isSource": false, "classifier": "ICAP_HEARTBEAT", "unitType": "seconds", "datapointType": "gauge", "id": "ICAP_HEARTBEAT"}, {"description": "", "isSource": true, "classifier": "NITS", "unitType": "kWh", "datapointType": "gauge", "id": "NITS", "name": "NITS"}, {"name": "NITS_HEARTBEAT", "isSource": false, "classifier": "NITS_HEARTBEAT", "unitType": "seconds", "datapointType": "gauge", "id": "NITS_HEARTBEAT"}, {"description": "Channel Vee Result with classifier VEE_RESULT", "isSource": false, "classifier": "VEE_RESULT", "unitType": "kWh", "datapointType": "interval", "id": "VEE_RESULT", "name": "Vee Result"}], "readOnly": false, "timezone": "America/New_York", "classifier": "ACCOUNT", "name": "NEPOOL_MECO_0219222018"}')
type(ds)

dict

In [27]:
ewx_dom = domain.EnergyworxDomain()
type(ewx_dom)
#ewx_dom.from_message(ds)

energyworx_public.base.EnergyworxDomain

In [25]:
domain.Datasource.from_message(ds)

AttributeError: type object 'Datasource' has no attribute 'attr_by_prop_name'

In [29]:
datasource = domain.Datasource()
type(datasource)
datasource.attr_by_prop_name

AttributeError: 'Datasource' object has no attribute 'attr_by_prop_name'

### from message function

In [74]:
class domain_jr(object):
    
    #__metaclass__ = EnergyworxDomainMeta

    #id = Property(name="id", required=True)
    #read_only = Property(name="readOnly", default=False)
    
    def __init__(self, data):
        for attr, value in data.items():
            setattr(self, attr, value)
            
ewx_jr = domain_jr(ds)
dir(ewx_jr)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'channels',
 'classifier',
 'description',
 'id',
 'name',
 'readOnly',
 'tags',
 'timezone']

### Tags instance

In [71]:
ewx_jr.tags

[{'validFrom': '1900-01-01T00:00:00.000000',
  'description': '',
  'tagLinks': [''],
  'version': '2019-12-03T19:16:17.000000',
  'tag': 'premise',
  'createdBy': 'System',
  'removed': False,
  'properties': [{'deleted': False,
    'value': 'MARLBOROUGH',
    'readOnly': False,
    'value_type': 'string',
    'key': 'city',
    'type': 'string'},
   {'deleted': False,
    'value': 'TIERPOINT-MA',
    'readOnly': False,
    'value_type': 'string',
    'key': 'name',
    'type': 'string'},
   {'deleted': False,
    'value': '01752',
    'readOnly': False,
    'value_type': 'string',
    'key': 'zip',
    'type': 'string'},
   {'deleted': False,
    'value': 'USA',
    'readOnly': False,
    'value_type': 'string',
    'key': 'country',
    'type': 'string'},
   {'deleted': False,
    'value': 'Unknown',
    'readOnly': False,
    'value_type': 'string',
    'key': 'county',
    'type': 'string'},
   {'deleted': False,
    'value': 'MA',
    'readOnly': False,
    'value_type': 'string'

### Channel instance

In [73]:
ewx_jr.channels

[{'description': 'Automatic generated channel',
  'isSource': True,
  'classifier': 'DELIVERY_IDR',
  'unitType': 'kWh',
  'datapointType': 'interval',
  'id': 'DELIVERY_IDR',
  'name': 'Delivery Idr'},
 {'description': 'Automatic generated channel',
  'isSource': True,
  'classifier': 'DELIVERY_SCALAR',
  'unitType': 'kWh',
  'datapointType': 'gauge',
  'id': 'DELIVERY_SCALAR',
  'name': 'Delivery Scalar'},
 {'description': 'No description',
  'isSource': False,
  'classifier': 'FORECAST',
  'unitType': 'kWh',
  'datapointType': 'interval',
  'id': 'FORECAST',
  'name': 'No name'},
 {'description': 'No description',
  'isSource': False,
  'classifier': 'LINEAR_INTERPOLATION',
  'unitType': 'kWh',
  'datapointType': 'interval',
  'id': 'LINEAR_INTERPOLATION',
  'name': 'No name'},
 {'description': 'No description',
  'isSource': False,
  'classifier': 'LIKE_DAY_SUBSTITUTION',
  'unitType': 'kWh',
  'datapointType': 'interval',
  'id': 'LIKE_DAY_SUBSTITUTION',
  'name': 'No name'},
 {'d

In [58]:
ewx_meta = EnergyworxDomainMeta('test', (), {})

In [50]:
kwargs = {}
for prop_name, value in ds.items():
    logger.debug('property {prop_name}, value: {value}'.format(prop_name=prop_name, value=value))
    attr = ewx_dom.attr_by_prop_name.get(prop_name) or (super(ewx_dom, ewx_dom).attr_by_prop_name.get(prop_name))
    if not attr:
        raise AttributeError('Property name {prop_name} can not be mapped to a domain attribute on domain class {class_name}'.format(prop_name=prop_name, class_name=cls.__name__))
    prop = ewx_dom.prop_by_attr.get(attr) or (super(ewx_dom, ewx_dom).prop_by_attr.get(attr))
    if not prop:
        raise AttributeError('Attribute {attr_name} can not be mapped to a property instance on domain class {class_name}'.format(attr_name=attr, class_name=cls.__name__))

    if isinstance(prop, StructuredProperty):
        if prop.repeated:
            kwargs.update({attr: [prop.domain_class.from_message(v) for v in value]})
        else:
            kwargs.update({attr: prop.domain_class.from_message(value)})
    elif isinstance(prop, EnumProperty):
        kwargs.update({attr: str_to_enum(prop.enum_type, value)})
    elif isinstance(prop, DateTimeProperty):
        kwargs.update({attr: parse(value)})
    else:
        kwargs.update({attr: value})
    

AttributeError: 'EnergyworxDomain' object has no attribute 'attr_by_prop_name'

In [82]:
print(type(ds))
pp.pprint(ds)

<class 'dict'>
{'channels': [{'classifier': 'DELIVERY_IDR',
               'datapointType': 'interval',
               'description': 'Automatic generated channel',
               'id': 'DELIVERY_IDR',
               'isSource': True,
               'name': 'Delivery Idr',
               'unitType': 'kWh'},
              {'classifier': 'DELIVERY_SCALAR',
               'datapointType': 'gauge',
               'description': 'Automatic generated channel',
               'id': 'DELIVERY_SCALAR',
               'isSource': True,
               'name': 'Delivery Scalar',
               'unitType': 'kWh'},
              {'classifier': 'FORECAST',
               'datapointType': 'interval',
               'description': 'No description',
               'id': 'FORECAST',
               'isSource': False,
               'name': 'No name',
               'unitType': 'kWh'},
              {'classifier': 'LINEAR_INTERPOLATION',
               'datapointType': 'interval',
               'descripti

### Scratch Work
try making class from API

In [30]:
#dir(base.Property())

In [31]:
__meta__ = EnergyworxDomainMeta
__meta__

__main__.EnergyworxDomainMeta

### Metadomain Class

In [45]:
new_class = super(EnergyworxDomainMeta, dom).__new__(meta_dom, name, bases, dct)

TypeError: super(type, obj): obj must be an instance or subtype of type

### Domain Class

In [52]:
dom = base.EnergyworxDomain()